In [1]:
import rioxarray
import geopandas
import os
import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle

In [ ]:
out_dir = "data/_grace_tiff"
os.makedirs(out_dir, exist_ok=True)
geodf = geopandas.read_file("data/aoi.geojson")
nc = rioxarray.open_rasterio("data/GRCTellus.JPL.200204_202202.GLO.RL06M.MSCNv02CRI.nc", decode_times=False)[0]
grace_start_date = datetime.datetime(2002, 1, 1)

In [ ]:
nc

Extract tiffs from nc file directly into pandas dataframe. Data was upsampled to 8x13 to match others' data dimensions.

In [ ]:
nc = nc.rio.write_crs("EPSG:4326")
print("Extracting tifs...")
df = pd.DataFrame(columns = ["date", "value", "mask"])
dates = []
for lwe in tqdm(nc["lwe_thickness"]):
    lwe = lwe.rio.clip(geodf.geometry.values, geodf.crs, all_touched=True)
    lwe = lwe.rio.write_crs("EPSG:4326")
    new_width = 13
    new_height = 8

    lwe_upsampled = lwe.rio.reproject(
        lwe.rio.crs,
        shape=(new_height, new_width),
    )
    lwe_datetime = grace_start_date + datetime.timedelta(days=int(lwe.time.values))
    lwe_upsampled.values[lwe_upsampled.values < -99990] = np.nan
    mean = lwe_upsampled.mean(skipna=True)
    masked = lwe_upsampled.to_masked_array()
    dates.append(lwe_datetime.strftime('%Y-%m-%d'))
    df = df.append({'date': lwe_datetime.strftime('%Y-%m-%d'), 'value': masked.data, 'mask': np.invert(masked.mask)}, ignore_index=True)

In [10]:
df.to_pickle("data/grace_df.pickle")

Save grace dates into the file. This file is mandatory for other scripts to run.

In [7]:
with open('data/grace_dates.pickle', 'wb') as f:
    pickle.dump(dates, f)